##Post2Caption

Adam Khakhar

akhakhar@wharton.upenn.edu



##Motivation

Image captioning has been researched intensely in the past 10 years. Recurrent neural models for image captioning usually employ an encoder-decodr gramework with an attention mechanism. In most of the image captioning research, the dataset includes captions directly tied to images: the captions serve as a description or story of the image. However, in social media posts, it is common for captions to be styled as a joke or a statement, rather than a vanilla description of the image. In this paper, I intend to research applications of a recurrent neural model for image captioning where the caption is not necessarily a description of the image or the objects in the image.



##Architecture and Hyperparameter Selection Summary

The goal of this model is to generate an Instagram styled caption for a given post (image). To accomplish this, I will use an encoder decoder combination of a Convolutional Neural Network and Recurrent Neural Network with attention. I will use a pretrained CNN (InceptionV3) to generate a vector for the image. I will feed this vector into the RNN with attention and Gated Recurrent Unit to generate a caption.

##Acknowledgements and References

This project was supported by the University of Pennsylvania School of Engineering and Applied Sciences, Computer Science Department: CIS 545: Big Data Analytics taught by Professor Zachary Ives, PhD. The works presented were compiled by Adam Khakhar. Some parts of the notebook include attributions to various papers from which Adam Khakhar has adapted, with some parts copied directly.
 

*   Justin Johnson, Andrej Karpathy, Li Fei-Fei; The IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2016, pp. 4565-4574
*   Steven J. Rennie, Etienne Marcheret, Youssef Mroueh, Jerret Ross, Vaibhava Goel; The IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2017, pp. 7008-7024
*   Andrej Karpathy, Li Fei-Fei; The IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2015, pp. 3128-3137
*   Karpathy, A. and Li, F., n.d. Automated Image Captioning With Convnets And Recurrent Nets. https://cs.stanford.edu/people/karpathy/sfmltalk.pdf
*   Karpathy, A, Neuraltalk2, https://github.com/karpathy/neuraltalk2
*   Daoust, Mark (Developer Programs Engineer @ Tensorflow Google) Image Captioning Notebook: https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/image_captioning.ipynb 




##Exploration of Data

This paper uses data scraped from Instagram including the post image and the caption of the post. The data was scraped by Prithvi Jaunjale and is freely available here: https://www.kaggle.com/prithvijaunjale/instagram-images-with-captions.
I have cleaned the data, merged folders, and have provided the clean data: "INSERT LINK".
In the following module, I will clean and explore the data.